# News Feed to DataBase based on Customer specific information

In [10]:
import feedparser,psycopg2,json,math,pytz
import pandas as pd
import numpy as np
import mysql.connector
from googlesearch import search 
from datetime import datetime
from dateutil import tz
# global str
import requests
from bs4 import BeautifulSoup

In [11]:
# read file
with open('configurationfile_Test.json', 'r') as myfile:
    data=myfile.read()

# parse file
objs = json.loads(data)

In [12]:
#Connection  to PostGresql
# conn = psycopg2.connect(host=str(objs[0]['host']),database=str(objs[0]['database']), user=str(objs[0]['user']), password=str(objs[0]['password']))

In [13]:
conn = psycopg2.connect("dbname=postgres user=postgres password=Msjobs@123")

In [3]:
#Connecting to MySQL
# conn = mysql.connector.connect(user=str(objs[0]['user']), password=str(objs[0]['password']),host=str(objs[0]['host']),database=str(objs[0]['database']))

In [14]:
def insert_data(schema,table,Serial_no,Time_Stamp,Client,News,Url,Site,Published_time):
    cur = conn.cursor()
    sql_command = "INSERT INTO {}.{}(Serial_no,Time_Stamp,Client_,News,Url,Site,published_Time) VALUES('{}','{}','{}','{}','{}','{}','{}');".format(str(schema), str(table),str(Serial_no),str(Time_Stamp),str(Client),str(News),str(Url),str(Site),str(Published_time))
    cur.execute(sql_command)
    conn.commit()
    cur.close() 

# Getting the maximum serial number from table
def gettingMax(obj):
    cur = conn.cursor()
    sql_command = "SELECT MAX(serial_no) FROM "+str(obj['schema'])+"."+str(obj['table'])
    data = pd.read_sql(sql_command, conn)
    i_nan = lambda x : 0 if math.isnan(x) else x # Checking if nan is present and assigning 0 to it
    data = i_nan(data.max()[0])
    conn.commit()
    cur.close()
    return data

# checking if the string is present in sentence
def check_string(strings,words):
    for word in words:
        if (word.lower() in strings.lower()):
            return True
    return False

# getting the complete web page into python
def parseRSS(rss_url):
    return feedparser.parse(rss_url)

# Getting the site name from Url
def sitename(Url):
    x = Url.split("//")[1].split("/")[0]
    x = x.replace("www.", "")
    x = x.replace(".com", "")
    x = x.split(".")[0]
    x = x[0]+x[1:].lower()
    return x

# getting the News article and URL from webpage
def getHeadlines(key,rss_url,obj):
    headlines = []
    Insights = []
    feed = parseRSS(rss_url)
    icount = 0
    i = gettingMax(obj)
    for newsitem in feed['items']:
        Serial_no = i+1
        News = newsitem['title'].replace("'", "").replace("&#39", "").replace(";", "")
        Url = newsitem['links'][0]['href']
        Client = key
        date_ = newsitem['published']
        try:
            timestampStr = datetime.strptime(date_,"%a, %d %b %Y %H:%M:%S %z")
            Published_time =str(timestampStr.astimezone(tz.tzutc())).split("+")[0]
        except ValueError:
            timestampStr = datetime.strptime(date_,"%a, %d %b %Y %H:%M:%S %Z")
            Published_time =str(timestampStr.astimezone(tz.tzutc())).split("+")[0]
        Site = newsitem['source']['title'].replace("'", "")
        Time_Stamp = str(datetime.now())
        words = list(obj['words'])
        if check_string(News,words):
            insert_data(str(obj['schema']),str(obj['table']),Serial_no,Time_Stamp,Client,News,Url,Site,Published_time)
            i = i+1 
            Insights.append(News)
            Insights.append(Url)
            icount =icount+1
            if icount >= obj['news_limit']:
                break
    return headlines

# getting the News article and URL from webpage from rss
def getHeadlinesrss(key,rss_url,obj):
    headlines = []
    Insights = []
    feed = parseRSS(rss_url)
    icount = 0
    i = gettingMax(obj)
    for newsitem in feed['entries']:
        Serial_no = i+1
        News = newsitem.title.replace("<b>","").replace("</b>","").replace("&#39", "").replace(";", "")
        Url = newsitem.link
        Client = key
#         Published_time = newsitem['published']
        date_ = newsitem['published'].replace("Z","")
        timestampStr = datetime.strptime(date_, '%Y-%m-%dT%H:%M:%S')
#         Published_time = timestampStr.strftime("%Y-%m-%d %X.%f")
        Published_time = str(timestampStr.astimezone(tz.tzutc())).split("+")[0]
        Site = key
        Time_Stamp = str(datetime.now())
        words = list(obj['words'])
        if check_string(News,words):
            insert_data(str(obj['schema']),str(obj['table']),Serial_no,Time_Stamp,Client,News,Url,Site,Published_time)
            i = i+1 
            Insights.append(News)
            Insights.append(Url)
            icount =icount+1
            if icount >= obj['news_limit']:
                break
    return headlines

In [15]:
# Truncating the table
def TruncateTable(obj):
    cur = conn.cursor()
    sql_command = "delete from "+str(obj['schema'])+"."+str(obj['table'])
    cur.execute(sql_command)
    conn.commit()
    cur.close()  

In [16]:
# search word
def ClientSpecificNews(obj):
    Insights = []
    query = str(obj['query'])    
    for j in search(query, tld="co.in", num=10, stop=obj['news_limit'], pause=3):
        Insights.append(j)
    # Generating client news     
    Insights_specific = []
    for insight in Insights:
        word1 = list(obj['word1'])
        word2 = list(obj['word2'])
        if check_string(insight,word1) and check_string(insight,word2):
            spl_word = '/'
            if spl_word in insight:
                x = insight.split("/")
                if (len(x[len(x)-1])) > (len(x[len(x)-2])):
                    res = x[len(x)-1]
                    res = res.replace("-", " ") 
                    Insights_specific.append(res)
                    Insights_specific.append(insight)
                else:
                    res = x[len(x)-2]
                    res = res.replace("-", " ") 
                    Insights_specific.append(res)
                    Insights_specific.append(insight)    
    # Injesting into Database
    if len(Insights_specific)%2 ==0:
        i =  0
        icount = gettingMax(obj) +1
        while i < len(Insights_specific):
            Serial_no = icount
            Time_Stamp = str(datetime.now())
            Client = str(obj['Client'])
#             News = Insights_specific[i].replace("&#39", "").replace(";", "")
            Url = Insights_specific[i+1]
          
            page = requests.get(Url)
            soup = BeautifulSoup(page.content, 'html.parser')
            try:
                print(soup.head.title)
                if "title" in str(soup.head.title):
                    News = str(soup.head.title).split("</")[0].split(">")[1].strip().replace("'", "")
                else:
                    News = Insights_specific[i].replace("&#39", "").replace(";", "")                    
            except AttributeError:
                News = Insights_specific[i].replace("&#39", "").replace(";", "")

#             News = str(soup.head.title).split("</")[0].split(">")[1].strip().replace("'", "")
            
#             Published_time = str(datetime.now())
            Published_time =str(datetime.now().astimezone(tz.tzutc())).split(".")[0]
            Site = sitename(Url)
            Site = Site[0].upper()+Site[1:]
            icount = icount+1
            insert_data(str(obj['schema']),str(obj['table']),Serial_no,Time_Stamp,Client,News,Url,Site,Published_time)
            i = i+2  

In [17]:
for obj in objs:
    if   str(obj['truncate']) == 'Yes' and str(obj['query']) == 'generic' and str(obj['rss_type'])=='No':
        TruncateTable(obj)
        getHeadlines(str(obj['rss']),str(obj['url']),obj)
    elif str(obj['truncate']) != 'Yes' and str(obj['query']) == 'generic' and str(obj['rss_type'])=='No':
        getHeadlines(str(obj['rss']),str(obj['url']),obj)
    elif str(obj['truncate']) != 'Yes' and str(obj['query']) == 'generic' and str(obj['rss_type'])=='Yes':
        getHeadlinesrss(str(obj['rss']),str(obj['url']),obj)       
    elif str(obj['truncate']) == 'Yes' and str(obj['query']) != 'generic':
        TruncateTable(obj)
        ClientSpecificNews(obj)
    else:  
        ClientSpecificNews(obj)
        


<title data-rh="true">Live Coronavirus News Updates - The New York Times</title>
<title>Coronavirus live updates: Conservatives protest restrictions in Maryland, Texas; Florida beaches reopen to crowds - The Washington Post</title>
<title>Coronavirus update: Protests; Trump 'liberate' tweets; Texas rules</title>


SSLError: HTTPSConnectionPool(host='www.dshs.state.tx.us', port=443): Max retries exceeded with url: /coronavirus/ (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))

In [1]:
import pytz, datetime
from datetime import datetime
from dateutil import tz

utc_local = str(datetime.now().astimezone(tz.tzutc())).split(".")[0]
# utc_local = str(datetime.now().astimezone(tz.tzutc()))
print(utc_local)
print(datetime.now())

date_ = 'Wed, 25 Mar 2020 20:56:17 GMT'
timestampStr = datetime.strptime(date_,"%a, %d %b %Y %H:%M:%S %Z")
# Published_time = timestampStr.strftime("%Y-%m-%d %X")
utc_edit = str(timestampStr.astimezone(tz.tzutc())).split("+")[0]
print(utc_edit)


2020-03-29 13:43:47
2020-03-29 08:43:47.641255
2020-03-26 01:56:17
